# Mettabook

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

print("Setup complete! Auto-reload enabled.")

## Fetch Metrics

In [ ]:
from metta.app_backend.clients.scorecard_client import ScorecardClient

client = ScorecardClient()

In [ ]:
result = await client.sql_query("""WITH policy_performance AS (
    SELECT 
        e.eval_category,
        e.primary_policy_id,
        AVG(eam.value) as avg_reward
    FROM episodes e
    JOIN episode_agent_metrics eam ON e.internal_id = eam.episode_internal_id
    join policies p on p.id = e.primary_policy_id
    WHERE eam.metric = 'reward'
        AND e.eval_category IS NOT NULL
        AND p.created_at > now() - '5d'::interval
        AND p.url ilike 'wandb://%'
    GROUP BY e.eval_category, e.primary_policy_id
),
ranked_policies AS (
    SELECT 
        eval_category,
        primary_policy_id,
        avg_reward,
        ROW_NUMBER() OVER (PARTITION BY eval_category ORDER BY avg_reward DESC) as rank
    FROM policy_performance
),
ranked_training_runs AS (
SELECT 
    rp.eval_category,
    rp.primary_policy_id,
    p.name as policy_name,
    rp.avg_reward,
    rp.rank,
    tr.id as training_run_id,
    tr.name as training_run_name,
    tr.status as training_run_status,
    tr.created_at as training_run_created_at
FROM ranked_policies rp
JOIN policies p ON rp.primary_policy_id = p.id
JOIN epochs ep ON p.epoch_id = ep.id
JOIN training_runs tr ON ep.run_id = tr.id
WHERE rp.rank <= 10
ORDER BY rp.eval_category, rp.rank
)
select distinct(training_run_name) from ranked_training_runs;
""")
training_run_names = [a[0] for a in result.rows]


In [ ]:
print(training_run_names[:2])
print(', '.join(training_run_names))

In [ ]:
async def get_all_results(client, training_run_names: list[str]) -> list:
      all_results = []
      page = 1
      page_size = 1000

      while True:
          print(f"Fetching page {page}")
          result = await client.sql_query(f"""
              SELECT 
                  p.name as policy_name,
                  e.eval_name,
                  AVG(eam.value) as avg_reward
              FROM training_runs tr
              JOIN epochs ep ON ep.run_id = tr.id
              JOIN policies p ON p.epoch_id = ep.id
              JOIN episodes e ON e.primary_policy_id = p.id
              JOIN episode_agent_metrics eam ON eam.episode_internal_id = e.internal_id
              WHERE tr.name IN ({', '.join(("'" + name + "'" for name in training_run_names))})
                  AND eam.metric = 'reward'
              GROUP BY p.id, p.name, e.eval_name
              ORDER BY p.id, e.eval_name
              LIMIT {page_size} OFFSET {(page - 1) * page_size}
          """)

          if not result.rows:
              break

          all_results.extend(result.rows)

          if len(result.rows) < page_size:
              break

          page += 1

      return all_results


In [ ]:
results = await get_all_results(client, training_run_names)
print(len(results))


In [ ]:
import pandas as pd

def create_scorecard_dataframe(data: list[list]) -> pd.DataFrame:
    df_data = {}
    for policy_name, eval_name, score in data:
        if policy_name not in df_data:
            df_data[policy_name] = {}
        df_data[policy_name][eval_name] = score

    return pd.DataFrame(df_data).fillna(float('nan'))


In [ ]:
df = create_scorecard_dataframe(results)